In [54]:
import cv2 
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from sklearn.model_selection import train_test_split

torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.conv_base = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2, bias=False),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.fc_base = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
            
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x):
        x = self.conv_base(x)
        x = x.view(x.size(0), 256*6*6)
        x = self.fc_base(x)
        return x

In [ ]:
image = cv2.imread(os.path.join("C:/Users/ericp/Desktop/pixiv/data/normal/", "72390335_p0_square1200.jpg"))
image = cv2.resize(image, (224,224)) 
plt.imshow(image)

In [58]:
images_n = []
for file in os.listdir("C:/Users/ericp/Desktop/pixiv/data/normal"):
    image = cv2.imread(os.path.join("C:/Users/ericp/Desktop/pixiv/data/normal", file))
    image = cv2.resize(image, (224, 224)) 
    #blue_image = image[:,:,0]
    #green_image = image[:,:,1]
    #red_image = image[:,:,2]
    images_n.append(list((np.transpose(image)/255.0).astype('float64')))
images_n = np.array(images_n)
print(images_n.shape)

(7207, 3, 224, 224)


In [59]:
images_r = []
for file in os.listdir("C:/Users/ericp/Desktop/pixiv/data/r18"):
    image = cv2.imread(os.path.join("C:/Users/ericp/Desktop/pixiv/data/r18", file))
    image = cv2.resize(image, (224, 224)) 
    #blue_image = image[:,:,0]
    #green_image = image[:,:,1]
    #red_image = image[:,:,2]
    images_r.append(list((np.transpose(image)/255.0).astype('float64')))
images_r = np.array(images_r)
print(images_r.shape)

(3687, 3, 224, 224)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    np.concatenate((images_n, images_r), axis = 0), 
    np.concatenate((np.zeros(len(images_n)).astype('float64'), np.ones(len(images_r)).astype('float64')), axis = 0), 
    test_size=0.33)

In [64]:
class OurDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform
        
    def __len__(self):
        return len(self.X);
    
    def __getitem__(self, idx):
        item = torch.from_numpy(self.X[idx])
        label = torch.tensor(self.Y[idx], dtype=torch.long) 
        
        if self.transform:
            item = self.transform(item)
        
        return (item, label)

In [65]:
trainData = OurDataset(X_train,  y_train)
testData = OurDataset(X_test,  y_test)
trainLoader = DataLoader(trainData, batch_size=128, shuffle=True, num_workers=0)
testLoader = DataLoader(testData, batch_size=128, shuffle=False, num_workers=0)

In [ ]:
net = AlexNet()

optimizer = optim.SGD(net.parameters(), lr=0.1, weight_decay=0)
optimizer.zero_grad()

criterion = nn.CrossEntropyLoss()

def test(net, testLoader):
    net.eval()
    correct = 0
    with torch.no_grad():
        for (data,target) in testLoader:
            output = net(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        print("Test Accuracy: %f" % (100.*correct/len(testLoader.dataset)))

test(net, testLoader)

for epoch in range(10):
    net.train()
    for batch_idx, (data, target) in enumerate(trainLoader):
        pred = net(data)
        loss = criterion(pred, target)
        loss.backward()
        gn = 0
        for f in net.parameters():
            gn = gn + torch.norm(f.grad)
        print("E: %d; B: %d; Loss: %f; ||g||: %f" % (epoch, batch_idx, loss, gn))
        optimizer.step()
        optimizer.zero_grad()
    
    test(net, testLoader)

Test Accuracy: 0.000000
E: 0; B: 0; Loss: 6.899937; ||g||: 5.267571
E: 0; B: 1; Loss: 6.453909; ||g||: 12.648347
E: 0; B: 2; Loss: 1.373117; ||g||: 30.166508
E: 0; B: 3; Loss: 6.621624; ||g||: 6.944893
E: 0; B: 4; Loss: 5.642673; ||g||: 18.133717
E: 0; B: 5; Loss: 51.937539; ||g||: 747.667148
E: 0; B: 6; Loss: 6.413090; ||g||: 8.591099
E: 0; B: 7; Loss: 5.513053; ||g||: 22.229567
E: 0; B: 8; Loss: 134.822683; ||g||: 1433.180248
E: 0; B: 9; Loss: 6.582153; ||g||: 2.634963
E: 0; B: 10; Loss: 6.469398; ||g||: 3.857432
E: 0; B: 11; Loss: 6.397728; ||g||: 7.439692
E: 0; B: 12; Loss: 6.319406; ||g||: 6.231516
E: 0; B: 13; Loss: 6.113776; ||g||: 10.710555
E: 0; B: 14; Loss: 5.689651; ||g||: 7.021605
E: 0; B: 15; Loss: 5.364249; ||g||: 9.186271
E: 0; B: 16; Loss: 5.717992; ||g||: 3.759515
E: 0; B: 17; Loss: 4.811392; ||g||: 4.118954
E: 0; B: 18; Loss: 4.494277; ||g||: 3.532886
E: 0; B: 19; Loss: 4.034882; ||g||: 3.099444
E: 0; B: 20; Loss: 3.922315; ||g||: 1.990306
E: 0; B: 21; Loss: 3.331734;